## Preliminary Steps:

### Imports

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

### Credentials and Accessing the API

In [2]:
# Load API Credentials
with open('/Users/jalic/.secret/yelp_api.json') as f:
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

## API Extraction:

### Define Search

In [3]:
# Set API call parameters for favorite city and cuisine type
LOCATION = 'Memphis, TN'
TERM = 'Barbeque'

### Create a results-in-progress JSON file

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_Memphis_BBQ.json"
JSON_FILE

'Data/results_in_progress_Memphis_BBQ.json'

### Create a For Loop

In [5]:
# Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
# If it does not exist:
if file_exists == False:
    
    # CREATE ANY NEEDED FOLDERS
    #Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    # If JSON_FILE included a folder:
    if len(folder)>0:
        # Create the folder
        os.makedirs(folder, exist_ok=True)
        
    # INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    # Save an empty list
    with open(JSON_FILE, 'w') as f:
        json.dump([], f)

# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Memphis_BBQ.json already exists.


In [6]:
# Load previous results and use len of results for offset
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)
    
# Set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# Use our yelp_api variable's search_query method to perform our API call
results =yelp_api.search_query(location=LOCATION,
                              term=TERM,
                              offset=n_results)
# How many results total?
total_results = results['total']
total_results
# How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

- 20 previous results found.


49

### Add this page of results to .json file

In [7]:
# Join new results with old list with extend and save to file
previous_results.extend(results['businesses'])
with open (JSON_FILE, 'w') as f:
    json.dump(previous_results, f)

### Set up a progress bar in the for loop

In [8]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # Adds 200 ms pause
    time.sleep(.2)

  0%|          | 0/49 [00:00<?, ?it/s]

## After For Loop is Completed:

In [9]:
# Convert final results json file into a DataFrame
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,o_KvDfOLLlNKla4e7dwaTQ,central-bbq-memphis-3,Central BBQ,https://s3-media1.fl.yelpcdn.com/bphoto/p3Y4ia...,False,https://www.yelp.com/biz/central-bbq-memphis-3...,2314,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.0,"{'latitude': 35.13381, 'longitude': -90.05716}","[delivery, pickup]",$$,"{'address1': '147 E Butler Ave', 'address2': '...",+19016727760,(901) 672-7760,2475.360971
1,rB5uSDXgbj995hJ0wxZFmQ,paynes-bar-b-que-memphis,Payne's Bar-B-Que,https://s3-media2.fl.yelpcdn.com/bphoto/MiYRNH...,False,https://www.yelp.com/biz/paynes-bar-b-que-memp...,321,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 35.11817, 'longitude': -90.00306}",[delivery],$,"{'address1': '1762 Lamar Ave', 'address2': '',...",+19012721523,(901) 272-1523,2842.887948
2,GgPbCwGC-wci0v_YDe7bbQ,blues-city-cafe-memphis,Blues City Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/7nAbbr...,False,https://www.yelp.com/biz/blues-city-cafe-memph...,1729,"[{'alias': 'southern', 'title': 'Southern'}, {...",4.0,"{'latitude': 35.1402, 'longitude': -90.053379}","[delivery, pickup]",$$,"{'address1': '138 Beale St', 'address2': '', '...",+19015263637,(901) 526-3637,2343.523686
3,RBc7QurYPiFXyu7w5-RzXw,arnolds-bar-b-que-and-grill-memphis,Arnolds BAR-B-QUE & Grill,https://s3-media2.fl.yelpcdn.com/bphoto/eny1WB...,False,https://www.yelp.com/biz/arnolds-bar-b-que-and...,165,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 35.14292, 'longitude': -90.04546}",[delivery],$$,"{'address1': '337 Madison Ave', 'address2': 'S...",+19017300290,(901) 730-0290,1915.597797
4,4kqhpPpyT3ZAAoy9cO8SoA,charlie-vergos-rendezvous-memphis-3,Charlie Vergos' Rendezvous,https://s3-media3.fl.yelpcdn.com/bphoto/a4Ug5H...,False,https://www.yelp.com/biz/charlie-vergos-rendez...,2587,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",3.5,"{'latitude': 35.1433447203971, 'longitude': -9...","[delivery, pickup]",$$,"{'address1': '52 S 2nd St', 'address2': '', 'a...",+19015232746,(901) 523-2746,2381.831021


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
35,OYLkahSo1yrwi60mJImzxQ,bar-b-zoo-smokehouse-memphis,Bar B Zoo Smokehouse,https://s3-media2.fl.yelpcdn.com/bphoto/8-6SQ8...,False,https://www.yelp.com/biz/bar-b-zoo-smokehouse-...,1,"[{'alias': 'bbq', 'title': 'Barbeque'}]",5.0,"{'latitude': 35.14821758032184, 'longitude': -...",[],NaN,"{'address1': '2000 Prentiss Pl', 'address2': '...",,,3730.879500
36,Uqy3cJHMfc4crc4gpP8ugw,germantown-commissary-germantown,Germantown Commissary,https://s3-media3.fl.yelpcdn.com/bphoto/ySOToY...,False,https://www.yelp.com/biz/germantown-commissary...,1152,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.0,"{'latitude': 35.08646068169633, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '2290 S Germantown Rd', 'address2...",+19017545540,(901) 754-5540,20607.461749
37,haE2mFLcM3PSmGtIDQatcg,one-and-only-bbq-memphis-7,One & Only BBQ,https://s3-media3.fl.yelpcdn.com/bphoto/glzAY5...,False,https://www.yelp.com/biz/one-and-only-bbq-memp...,532,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 35.10068455653, 'longitude': -89....","[pickup, delivery]",$$,"{'address1': '1779 Kirby Pkwy', 'address2': 'S...",+19017513615,(901) 751-3615,17659.121212
38,OZjrUk-CJq7mVKPYTVfc0Q,flying-fish-memphis-2,Flying Fish,https://s3-media1.fl.yelpcdn.com/bphoto/_UXk4Z...,False,https://www.yelp.com/biz/flying-fish-memphis-2...,546,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 35.1420662668398, 'longitude': -9...",[delivery],$$,"{'address1': '105 S 2nd St', 'address2': '', '...",+19015228228,(901) 522-8228,2394.967028
39,t_UWaG7BMQungFcuMGgBXw,pollard-s-bbq-memphis,Pollard’s BBQ,https://s3-media4.fl.yelpcdn.com/bphoto/2Dn-FZ...,False,https://www.yelp.com/biz/pollard-s-bbq-memphis...,115,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 35.0241737365723, 'longitude': -9...",[delivery],$,"{'address1': '4560 Elvis Presley Blvd', 'addre...",+19013982987,(901) 398-2987,11888.945630


### Check for and remove any duplicate results 

In [10]:
# Check for duplicate ID's
final_df.duplicated(subset='id').sum()

0

### Save final DataFrame to a csv file

In [12]:
# Save the final results to a compressed csv
final_df.to_csv('Data/final_results_Memphis_BBQ.csv.gz', compression='gzip', index=False)